In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
#sns.set(style = 'white', color_code=True)
import warnings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.shape,test.shape
# test.plot(y='LoanAmount',x='Loan_Amount_Term',kind='scatter')

((614, 13), (367, 12))

In [3]:
train['source'] = 'train'
test['source'] = 'test'

In [4]:
data = pd.concat([train,test],ignore_index=False)
data.info()
# data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 981 entries, 0 to 366
Data columns (total 14 columns):
ApplicantIncome      981 non-null int64
CoapplicantIncome    981 non-null float64
Credit_History       902 non-null float64
Dependents           956 non-null object
Education            981 non-null object
Gender               957 non-null object
LoanAmount           954 non-null float64
Loan_Amount_Term     961 non-null float64
Loan_ID              981 non-null object
Loan_Status          614 non-null object
Married              978 non-null object
Property_Area        981 non-null object
Self_Employed        926 non-null object
source               981 non-null object
dtypes: float64(4), int64(1), object(9)
memory usage: 115.0+ KB


In [5]:
data.apply(lambda x: sum(x.isnull()))

ApplicantIncome        0
CoapplicantIncome      0
Credit_History        79
Dependents            25
Education              0
Gender                24
LoanAmount            27
Loan_Amount_Term      20
Loan_ID                0
Loan_Status          367
Married                3
Property_Area          0
Self_Employed         55
source                 0
dtype: int64

## Pivot Table

In [6]:
data['Gender'].fillna('Male',inplace=True)
#data['Dependents'].fillna('0',inplace=True)
data['Married'].fillna('Yes',inplace=True)
data['Self_Employed'].fillna('No',inplace=True)
#data['Credit_History'].fillna(1.0,inplace=True)
impute_grps = data.pivot_table(values=["LoanAmount"], index=["Gender","Married","Self_Employed"], aggfunc=np.mean)
# print(impute_grps.loc["Gender"].values[0])
# print(impute_grps)

In [7]:
#Create a new function:
def num_missing(x):
  return sum(x.isnull())

#iterate only through rows with missing LoanAmount
for i,row in data.loc[data['LoanAmount'].isnull(),:].iterrows():
    ind = tuple([row['Gender'],row['Married'],row['Self_Employed']])
    data.loc[i,'LoanAmount'] = impute_grps.loc[ind].values[0]

#Now check the #missing values again to confirm:
print(data.apply(num_missing, axis=0))

ApplicantIncome        0
CoapplicantIncome      0
Credit_History        79
Dependents            25
Education              0
Gender                 0
LoanAmount             0
Loan_Amount_Term      20
Loan_ID                0
Loan_Status          367
Married                0
Property_Area          0
Self_Employed          0
source                 0
dtype: int64


In [8]:
data.dtypes

ApplicantIncome        int64
CoapplicantIncome    float64
Credit_History       float64
Dependents            object
Education             object
Gender                object
LoanAmount           float64
Loan_Amount_Term     float64
Loan_ID               object
Loan_Status           object
Married               object
Property_Area         object
Self_Employed         object
source                object
dtype: object

## We have some object(String) Variable in our Dataset, Lets check their Categories

In [9]:
var = data[['Dependents','Education','Gender','Married','Property_Area','Self_Employed','Credit_History','Loan_Amount_Term']]
for v in var:
    print('\nFreq count of columns:%s'%v)
    print(data[v].value_counts())


Freq count of columns:Dependents
0     545
1     160
2     160
3+     91
Name: Dependents, dtype: int64

Freq count of columns:Education
Graduate        763
Not Graduate    218
Name: Education, dtype: int64

Freq count of columns:Gender
Male      799
Female    182
Name: Gender, dtype: int64

Freq count of columns:Married
Yes    634
No     347
Name: Married, dtype: int64

Freq count of columns:Property_Area
Semiurban    349
Urban        342
Rural        290
Name: Property_Area, dtype: int64

Freq count of columns:Self_Employed
No     862
Yes    119
Name: Self_Employed, dtype: int64

Freq count of columns:Credit_History
1.0    754
0.0    148
Name: Credit_History, dtype: int64

Freq count of columns:Loan_Amount_Term
360.0    823
180.0     66
480.0     23
300.0     20
240.0      8
84.0       7
120.0      4
60.0       3
36.0       3
12.0       2
350.0      1
6.0        1
Name: Loan_Amount_Term, dtype: int64


In [10]:
data.dtypes
# data[data['Loan_Amount_Term']<240]

ApplicantIncome        int64
CoapplicantIncome    float64
Credit_History       float64
Dependents            object
Education             object
Gender                object
LoanAmount           float64
Loan_Amount_Term     float64
Loan_ID               object
Loan_Status           object
Married               object
Property_Area         object
Self_Employed         object
source                object
dtype: object

In [11]:
data['Total_Income'] = data['ApplicantIncome'] + data['CoapplicantIncome']
#data['Total_Income_Log'] = np.log(data['Total_Income'])
data['LoanAmount'].fillna(data['LoanAmount'].mean(),inplace=True)
#To remove outliers in LoanAmount 
#data['LoanAmount_log'] = np.log(data['LoanAmount'])
#data['LoanAmount_log'].fillna(data['LoanAmount_log'].mean(),inplace=True)
data['Loan_Amount_Term'].fillna(data['Loan_Amount_Term'].mean(),inplace=True)
#data['EMI'] = data['LoanAmount_log']/data['Loan_Amount_Term']
data['EMI'] = data['LoanAmount']/data['Loan_Amount_Term']
#data['EMI'] = (data['LoanAmount'] * 0.095 * ((1+0.095) ** data['Loan_Amount_Term'])) \
                     # /(1.095 ** data['Loan_Amount_Term'] -1)
#data['Ratio'] = data['LoanAmount_log']/data['Total_Income']
#data['Ratio'] = data['LoanAmount']/data['Total_Income']
# data[['EMI','Loan_Status','LoanAmount','Loan_Amount_Term']][data['LoanAmount']<100]
dep_map = {'0':'0','1':'1','2':'2','3+':'3'}
data['Dependents'] = data['Dependents'].map(dep_map)
data['not_self_graduate_married'] = data.apply(lambda x: 1 if (x['Self_Employed'] == 'No') & (x['Married'] == 'Yes') \
                                                       & (x['Education'] == 'Graduate' ) & (x['Dependents'] not in ['2','3']) else 0, axis=1)
data['Ratio'] = data['LoanAmount']/data['Total_Income']
data['R_6_Unmarried'] = data.apply(lambda x: 1 if (x['Ratio'] > 6) & (x['Married'] == 'No') else 0, axis=1)
data['male_6000_not_graduate'] = data.apply(lambda x: 1 if (x['ApplicantIncome'] < 6000) & (x['Gender'] == 'Male') & (x['Education'] == 'Not Graduate') else 0, axis=1)
data['female_self'] = data.apply(lambda x: 1 if (x['Gender'] == 'Female') & (x['Self_Employed'] == 'Yes') else 0, axis=1)

In [12]:
# data.drop('ApplicantIncome',axis=1,inplace=True)
# data.drop('CoapplicantIncome',axis=1,inplace=True)
data.dtypes

ApplicantIncome                int64
CoapplicantIncome            float64
Credit_History               float64
Dependents                    object
Education                     object
Gender                        object
LoanAmount                   float64
Loan_Amount_Term             float64
Loan_ID                       object
Loan_Status                   object
Married                       object
Property_Area                 object
Self_Employed                 object
source                        object
Total_Income                 float64
EMI                          float64
not_self_graduate_married      int64
Ratio                        float64
R_6_Unmarried                  int64
male_6000_not_graduate         int64
female_self                    int64
dtype: object

In [13]:
#data['Gender'].fillna('Male',inplace=True)
data['Dependents'].fillna('0',inplace=True)
data['Married'].fillna('Yes',inplace=True)
#data['Self_Employed'].fillna('No',inplace=True)
data['Credit_History'].fillna(1.0,inplace=True)
data.drop('Gender',axis=1,inplace=True)
data.drop('Self_Employed',axis=1,inplace=True)
# data.drop('LoanAmount',axis=1,inplace=True)
# data.drop('Loan_Amount_Term',axis=1,inplace=True)
#data.drop('Dependents',axis=1,inplace=True)
#data.drop('Married',axis=1,inplace=True)

In [14]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#var_to_encode = ['Dependents','Education','Gender','Married','Property_Area','Self_Employed']
var_to_encode = ['Married','Dependents','Education','Property_Area']
#var_to_encode = ['Married','Education','Property_Area']
for col in var_to_encode:
    data[col] = le.fit_transform(data[col])

In [15]:
data = pd.get_dummies(data,columns=var_to_encode)

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 981 entries, 0 to 366
Data columns (total 26 columns):
ApplicantIncome              981 non-null int64
CoapplicantIncome            981 non-null float64
Credit_History               981 non-null float64
LoanAmount                   981 non-null float64
Loan_Amount_Term             981 non-null float64
Loan_ID                      981 non-null object
Loan_Status                  614 non-null object
source                       981 non-null object
Total_Income                 981 non-null float64
EMI                          981 non-null float64
not_self_graduate_married    981 non-null int64
Ratio                        981 non-null float64
R_6_Unmarried                981 non-null int64
male_6000_not_graduate       981 non-null int64
female_self                  981 non-null int64
Married_0                    981 non-null float64
Married_1                    981 non-null float64
Dependents_0                 981 non-null float64
Dependen

In [17]:
train = data.loc[data['source']=='train']
test = data.loc[data['source']=='test']

In [18]:
train.drop('source',axis=1,inplace=True)
test.drop(['source','Loan_Status'],axis=1,inplace=True)

In [19]:
train.to_csv('train_modified.csv',index=False)
test.to_csv('test_modified.csv',index=False)